# Medium exploration
This serves as notebook to look at different metabolites or reactions or ... in the GEMs present in this repository. 

In [1]:
# imports
import cobra
from libsbml import *
import refinegems as rg
import pandas as pd

RefineGEMs provides a couple of functions that make life a bit easier, so they will be used here. I still loaded cobra and libSBML to ensure that I have access to everything.

First we load our model with cobrapy.

In [2]:
model = rg.load.load_model_cobra('../models/Cstr_14.xml')

Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled


### Focus: Protocatechuic acid (PCA)
PCA is part of the CGXII medium which is of interest at the moment because this might be the minimal medium we need for more laboratory experiments.

In [4]:
model.metabolites.get_by_id('34dhbz_c')

Metabolite identifier,34dhbz_c
Name,"3,4-Dihydroxybenzoate"
Memory address,0x07f98b2c1dbb0
Formula,C7H5O4
Compartment,c
In 2 reaction(s),"PCADYOX, VNTDM"


Now we have a closer look at the two reactions that 34dhbz participates in.

In [5]:
model.reactions.get_by_id('PCADYOX')

Reaction identifier,PCADYOX
Name,
Memory address,0x07f9870e72fa0
Stoichiometry,"34dhbz_c + o2_c --> CCbuttc_c + 2.0 h_c 3,4-Dihydroxybenzoate + O2 O2 --> Cis,cis-Butadiene-1,2,4-tricarboxylate + 2.0 H+"
GPR,
Lower bound,0.0
Upper bound,1000.0


In [6]:
model.reactions.get_by_id('VNTDM')

Reaction identifier,VNTDM
Name,
Memory address,0x07f9889d44c70
Stoichiometry,"h_c + nadh_c + o2_c + vanlt_c --> 34dhbz_c + fald_c + h2o_c + nad_c H+ + Nicotinamide adenine dinucleotide - reduced + O2 O2 + Vanillate --> 3,4-Dihydroxybenzoate + Formaldehyde + H2O H2O + Nicotinamide adenine dinucleotide"
GPR,
Lower bound,0.0
Upper bound,1000.0


In strain 14 VNTDM produces 34dhbz, PCADYOX consumes 34dhbz. There exists no uptake reaction. In the other strains (15, 16, 17) 34dhbz is not even present.

In [7]:
len(cobra.medium.find_boundary_types(model, 'sink'))
cobra.medium.find_boundary_types(model, 'sink')
cobra.flux_analysis.loopless_solution(model)

minimal = cobra.medium.minimal_medium(model).to_dict()

In [8]:
cgxlab = rg.load.load_medium_from_db('/Users/baeuerle/Organisation/Masterarbeit/refinegems/data/media_db.csv', 'CGXlab')['BiGG_EX']
len(cgxlab)

20

In [9]:
with model:
    medium = model.medium
    new_medium = {i: 10.0 for i in cgxlab}
    new_medium.pop('EX_ni2_e')
    new_medium.pop('EX_na1_e')
    new_medium.pop('EX_34dhbz_e')
    try:
        if (new_medium['EX_o2_e'] == 10.0):
            new_medium['EX_o2_e'] = 20.0
    except KeyError:
        print('No Oxygen Exchange Reaction')
        pass
    model.medium = new_medium
    sol = model.optimize()
    print(model.medium)
    secretion = []
    uptake = []
    for index, value in sol.fluxes.items():
        if value > 0:
            secretion.append(index)
        if "EX" in index:
            if value < 0:
                uptake.append(index)
sol

{'EX_btn_e': 10.0, 'EX_ca2_e': 10.0, 'EX_cl_e': 10.0, 'EX_cu2_e': 10.0, 'EX_fe2_e': 10.0, 'EX_glc__D_e': 10.0, 'EX_h2o_e': 10.0, 'EX_h_e': 10.0, 'EX_k_e': 10.0, 'EX_mg2_e': 10.0, 'EX_mn2_e': 10.0, 'EX_nh4_e': 10.0, 'EX_o2_e': 20.0, 'EX_pi_e': 10.0, 'EX_so4_e': 10.0, 'EX_urea_e': 10.0, 'EX_zn2_e': 10.0}


,fluxes,reduced_costs
12DGR120tipp,0.0,0.000000e+00
12DGR140tipp,0.0,0.000000e+00
12DGR141tipp,0.0,0.000000e+00
12DGR160tipp,0.0,0.000000e+00
12DGR161tipp,0.0,0.000000e+00
...,...,...
EX_zn2_e,0.0,0.000000e+00
Growth,0.0,1.055350e-16
ATPM,0.0,5.009806e-18
APCPT,0.0,-1.261869e-30


Determine unused metabolites.

In [10]:
not_used = list(set(cgxlab)-set(uptake))

In [11]:
new = list(set(cgxlab)-set(not_used))
with model:
    med = model.medium
    new_medium = {i: 10.0 for i in new}
    new_medium['EX_o2_e'] = 20.0
    model.medium = new_medium
    sol = model.optimize()
sol.objective_value

0.0

## Focus: missing metabolites for growth on CGXlab

### 1. L-cysteinylglycine (`cgly`)

In [12]:

model = rg.load.load_model_cobra('../models/Cstr_16.xml')

In [13]:
for rea in model.metabolites.get_by_id('cgly_c').reactions:
    print(rea)

DIPEPabc15: atp_c + cgly_e + h2o_c --> adp_c + cgly_c + h_c + pi_c
AMPTASECG: cgly_c + h2o_c --> cys__L_c + gly_c
CGLYabcpp: atp_c + cgly_p + h2o_c --> adp_c + cgly_c + h_c + pi_c


`cys__L_c` and `gly_c` are both part of the `Growth` reaction. `cgly_c` seems to be one way to produce both these metabolites. Maybe we can either find a different pathway to produce `cys__L_c` and `gly_c` or determine how `cgly_c` comes into the cell in the first place.

In [14]:
print('gly_c producing reactions')
for rea in model.metabolites.get_by_id('gly_c').reactions:
    if model.metabolites.get_by_id('gly_c') in rea.products and model.metabolites.get_by_id('cgly_c') not in rea.reactants and model.metabolites.get_by_id('gly_e') not in rea.reactants and model.metabolites.get_by_id('gly_p') not in rea.reactants:
        print(rea)

gly_c producing reactions
AMPEP11: gly_pro__L_c + h2o_c --> gly_c + pro__L_c
AMPEP9: gly_met__L_c + h2o_c <=> gly_c + met__L_c
SARCOX: h2o_c + o2_c + sarcs_c --> fald_c + gly_c + h2o2_c
THRA: thr__L_c --> acald_c + gly_c
AMPEP18: h2o_c + serglugly_c --> glu__L_c + gly_c + h_c + ser__L_c
GLYGLYCNc: glygly_c + h2o_c --> 2.0 gly_c
GLYTYRAP: gly_tyr__L_c + h2o_c <=> gly_c + tyr__L_c
THRA2: athr__L_c --> acald_c + gly_c
AMPTASEGGLN: glygln_c + h2o_c --> gln__L_c + gly_c
GLYPHEAP: gly_phe__L_c + h2o_c <=> gly_c + phe__L_c
AMPEP17: glyglygln_c + h2o_c --> gln__L_c + 2.0 gly_c
GLYCL_2: co2_c + mlthf_c + nadh_c + nh4_c --> gly_c + nad_c + thf_c
AMPEP16: h2o_c + lysglugly_c --> glu__L_c + gly_c + lys__L_c
GLYLEUAP: gly_leu__L_c + h2o_c <=> gly_c + leu__L_c
AMPTASEGM: glymet_c + h2o_c --> gly_c + met__L_c
AMPEP4: alagly_c + h2o_c <=> ala__L_c + gly_c
AMPEP3: gly_gln__L_c + h2o_c <=> gln__L_c + gly_c
AMPEP1: gly_asn__L_c + h2o_c <=> asn__L_c + gly_c
AMPEP10: gly_asp__L_c + h2o_c <=> asp__L_c + gly

In [15]:
print('cys__L producing reactions')
for rea in model.metabolites.get_by_id('cys__L_c').reactions:
    if model.metabolites.get_by_id('cys__L_c') in rea.products and model.metabolites.get_by_id('cgly_c') not in rea.reactants and model.metabolites.get_by_id('cys__L_e') not in rea.reactants and model.metabolites.get_by_id('cys__L_p') not in rea.reactants:
        print(rea)

cys__L producing reactions
CYSS: acser_c + h2s_c --> ac_c + cys__L_c


`cys__L` has only two reactions (14) / one reaction (15, 16, 17) which produce it. Both need `acser` to produce `cys__L`. See if O-Acetyl-L-serine can be added to a medium.

We can also directly add L-Cysteine to the medium to simulate addition of cgly. Lets see if that works in simulation.

In [16]:
with model:
    medium = model.medium
    new_medium = {i: 10.0 for i in cgxlab}
    new_medium.pop('EX_ni2_e')
    new_medium.pop('EX_na1_e')
    new_medium.pop('EX_34dhbz_e')
    new_medium.pop('EX_btn_e')
    new_medium['EX_cys__L_e'] = 10.0 # EX_cgly_e
    new_medium['EX_cobalt2_e'] = 10.0 
    new_medium['EX_pnto__R_e'] = 10.0
    try:
        if (new_medium['EX_o2_e'] == 10.0):
            new_medium['EX_o2_e'] = 20.0
    except KeyError:
        print('No Oxygen Exchange Reaction')
        pass
    model.medium = new_medium
    sol = model.optimize()
    secretion = []
    uptake = []
    for index, value in sol.fluxes.items():
        if value > 0:
            secretion.append(index)
        if "EX" in index:
            if value < 0:
                uptake.append(index)
sol, uptake

(<Solution 0.558 at 0x7f9871279f40>,
 ['EX_ca2_e',
  'EX_cl_e',
  'EX_cobalt2_e',
  'EX_cu2_e',
  'EX_cys__L_e',
  'EX_fe2_e',
  'EX_glc__D_e',
  'EX_k_e',
  'EX_mg2_e',
  'EX_mn2_e',
  'EX_o2_e',
  'EX_pi_e',
  'EX_pnto__R_e',
  'EX_so4_e',
  'EX_urea_e',
  'EX_zn2_e'])

Replacing `cgly` by `cys__L` works in the simulation -> test that in the lab.

### 2. β-nicotinamide D-ribonucleotide (`nmn`)

In [17]:
model = rg.load.load_model_cobra('../models/Cstr_17.xml')

In [18]:
for rea in model.metabolites.get_by_id('nmn_c').reactions:
    print(rea)

NMNDA: h2o_c + nmn_c --> nh4_c + nicrnt_c
NMNHYD: h2o_c + nmn_c --> pi_c + rnam_c
NADDP: h2o_c + nad_c --> amp_c + 2.0 h_c + nmn_c
NMNP: nmn_e --> nmn_c
NMNAT: atp_c + h_c + nmn_c --> nad_c + ppi_c


In [19]:
for rea in model.metabolites.get_by_id('ncam_c').reactions:
    print(rea)

PNP_1: pi_c + rnam_c <=> h_c + ncam_c + r1p_c
NNAM: h2o_c + ncam_c --> nac_c + nh4_c


In [20]:
for rea in model.metabolites.get_by_id('nac_c').reactions:
    print(rea)

NP1: h_c + nac_c + r1p_c --> nicrns_c + pi_c
NAPRT: h_c + nac_c + prpp_c <=> nicrnt_c + ppi_c
NNAM: h2o_c + ncam_c --> nac_c + nh4_c


`rnam`, `nac` and `ncam` only exist in the cytosol thus we cannot simulate the replacement of `nmn` by nicotinic acid (`nac`) but it is worth a try since it is included in the biosynthesis and available in the lab.

### 3. Beta-Alanine (`ala_B`)

In [21]:
model = rg.load.load_model_cobra('../models/Cstr_15.xml')

In [22]:
for rea in model.metabolites.get_by_id('ala_B_c').reactions:
    print(rea)

BALAt2r: ala_B_e + h_e --> ala_B_c + h_c
PANTS: ala_B_c + atp_c + pant__R_c --> amp_c + h_c + pnto__R_c + ppi_c
BALAt2pp: ala_B_p + h_p --> ala_B_c + h_c


`ala_B` produces `pnto__R` which we already test for the strains. So we replace `ala_B` by `pnto__R` and test if that works.

In [25]:
with model:
    medium = model.medium
    new_medium = {i: 10.0 for i in cgxlab}
    new_medium.pop('EX_ni2_e')
    new_medium.pop('EX_na1_e')
    new_medium.pop('EX_34dhbz_e')
    new_medium.pop('EX_btn_e')
    new_medium['EX_cys__L_e'] = 10.0 # replaces EX_cgly_e
    new_medium['EX_cobalt2_e'] = 10.0 
    new_medium['EX_pnto__R_e'] = 10.0 # replaces EX_ala_B_e
    try:
        if (new_medium['EX_o2_e'] == 10.0):
            new_medium['EX_o2_e'] = 20.0
    except KeyError:
        print('No Oxygen Exchange Reaction')
        pass
    model.medium = new_medium
    sol = model.optimize()
    secretion = {}
    uptake = {}
    for index, value in sol.fluxes.items():
        if value > 0:
            secretion[index] = value
        if "EX" in index:
            if value < 0:
                uptake[index] = value
uptake

{'EX_ca2_e': -0.002874433695242312,
 'EX_cl_e': -0.002874433695242312,
 'EX_cobalt2_e': -5.522447060984269e-05,
 'EX_cu2_e': -0.00039154149662378466,
 'EX_cys__L_e': -0.13601124417556934,
 'EX_fe2_e': -0.008020249866667453,
 'EX_glc__D_e': -10.0,
 'EX_k_e': -0.10779430091747025,
 'EX_mg2_e': -0.004790722825403854,
 'EX_mn2_e': -0.000381601091914013,
 'EX_o2_e': -20.0,
 'EX_pi_e': -0.5445646389707067,
 'EX_pnto__R_e': -0.00031809295071270546,
 'EX_so4_e': -0.002395637535054976,
 'EX_urea_e': -5.4379023608086365,
 'EX_zn2_e': -0.00018831544477956356}

## Focus: metabolites available in CGXII but not used by the models

1. Sodium (`na1`)

In [4]:
for rea in model.metabolites.get_by_id('na1_e').reactions:
    print(rea)

MFUMt8: fum_e + na1_e --> fum_c + na1_c
GLYt4: gly_e + na1_e --> gly_c + na1_c
NAt3_1: h_e + na1_c --> h_c + na1_e
NAt3_2: 2.0 h_e + na1_c --> 2.0 h_c + na1_e
GLUt4: glu__L_e + na1_e --> glu__L_c + na1_c
NAabc: atp_c + h2o_c + na1_e --> adp_c + h_c + na1_c + pi_c
r1143: asp__L_e + na1_e --> asp__L_c + na1_c
MALt4: mal__L_e + na1_e --> mal__L_c + na1_c
PROt4: na1_e + pro__L_e --> na1_c + pro__L_c
HMR_9610: na1_e + succ_e --> na1_c + succ_c
NAabcO: atp_c + h2o_c + na1_c --> adp_c + h_c + na1_e + pi_c
ALAt4: ala__L_e + na1_e --> ala__L_c + na1_c


`na1` seems to exist in the extracelullar region - why is there no uptake reaction? Sodium uptake is not as trivial as I thought.

2. Nickel (`ni2`)

Nickel is an essential trace element for bacteria as a cofactor of several well-characterized enzymes (Sevilla 2021).

In [6]:
for rea in model.metabolites.get_by_id('ni2_c').reactions:
    print(rea)

NI2uabcpp: atp_c + h2o_c + ni2_p --> adp_c + h_c + ni2_c + pi_c
NI2t4pp: h_p + k_p + ni2_c --> h_c + k_c + ni2_p
NI2t3pp: h_p + ni2_c --> h_c + ni2_p
NIabc: atp_c + h2o_c + ni2_e --> adp_c + h_c + ni2_c + pi_c
NI2abcpp: atp_c + h2o_c + ni2_c --> adp_c + h_c + ni2_p + pi_c


`ni2` is not directly used in the `Growth` reaction but is present in all three compartments.

3. Biotin (`btn`)

In [7]:
for rea in model.metabolites.get_by_id('btn_c').reactions:
    print(rea)

DTBTST: 2.0 amet_c + dtbt_c + s_c --> btn_c + 2.0 dad_5_c + 2.0 met__L_c
BTNTe: btn_c <=> btn_e
BTS2: cys__L_c + dtbt_c <=> ala__L_c + btn_c + 2.0 h_c
BTNt: atp_c + btn_e + h2o_c --> adp_c + btn_c + h_c + pi_c
BTS3r: dtbt_c + 2.0 s_c <=> btn_c + h2s_c + h_c
BTNt2i: btn_e + h_e --> btn_c + h_c


These are reactions also coming from _Mycobacterium tuberculosis H37Rv_ (DTBTST). Biotin biosynthesis seems to be 

4. PCA (`34dhbz`)

Seems to be carbon source for _C. glutamicum_

In [8]:
for rea in model.metabolites.get_by_id('34dhbz_c').reactions:
    print(rea)

VNTDM: h_c + nadh_c + o2_c + vanlt_c --> 34dhbz_c + fald_c + h2o_c + nad_c
PCADYOX: 34dhbz_c + o2_c --> CCbuttc_c + 2.0 h_c


In [9]:
model.reactions.get_by_id('VNTDM')

Reaction identifier,VNTDM
Name,
Memory address,0x07fbb11345e80
Stoichiometry,"h_c + nadh_c + o2_c + vanlt_c --> 34dhbz_c + fald_c + h2o_c + nad_c H+ + Nicotinamide adenine dinucleotide - reduced + O2 O2 + Vanillate --> 3,4-Dihydroxybenzoate + Formaldehyde + H2O H2O + Nicotinamide adenine dinucleotide"
GPR,
Lower bound,0.0
Upper bound,1000.0


In [10]:
model.reactions.get_by_id('PCADYOX')

Reaction identifier,PCADYOX
Name,
Memory address,0x07fbb50c04b50
Stoichiometry,"34dhbz_c + o2_c --> CCbuttc_c + 2.0 h_c 3,4-Dihydroxybenzoate + O2 O2 --> Cis,cis-Butadiene-1,2,4-tricarboxylate + 2.0 H+"
GPR,
Lower bound,0.0
Upper bound,1000.0


Seems to be part of the energy metabolism???